In [1]:
#import gradio as gr
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool,WebsiteSearchTool
from crewai_tools import tool
from datetime import datetime

#from docx import Document
from io import BytesIO
import base64

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/Albert/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] = os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

In [2]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0806',temperature=0)
#from langchain_openai import ChatOpenAI
#llm = ChatOpenAI(model="gpt-4o-0806", temperature=0)

In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
#web_rag_tool = WebsiteSearchTool()

In [5]:
# Define the Intelligent Medical Chatbot Agent
review_chatbot_agent = Agent(
    role="Research Analyst",
    goal="Investigate and summarize on the latest advancements for {topic}.",
    #tools=[scrape_tool, search_tool, web_rag_tool],  
    backstory=(
        "You are assisting a researcher to write an academic paper. You need to do literature Review in order to write 'background and related work' section of the paper."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool],
    llm=llm
)

review_chatbot_task = Task(
    description=(
        "Your task is to do literature Review for {topic}. Follow these guidelines:\n"
        "   - Utilize the search_tool or scrape_tool or web_rag_tool to gather relevant and up-to-date academic papers.\n"
        "   - Include the links of each paper, and the year of the paper."
        "   - Summarize the gathered information to write a literature review.\n"
    ),
    agent=review_chatbot_agent,
    expected_output=(
        "- Write background and related work which is a general summary for {topic} based on all the papers."
        "- For each paper, please give the link of paper, year of publication, and detailed summary of the paper. "
    )
)

In [6]:
# review_chatbot_crew = Crew(agents=[review_chatbot_agent], tasks=[review_chatbot_task], verbose=True)    
# inputs = {
#     "topic": "Interpretable Deep learning models for medical condition prediction using diagnosis, procedure, medication and lab codes",
# }
# result = review_chatbot_crew.kickoff(inputs=inputs)

In [7]:
import re
import base64
from PIL import Image
import io

def chat(message, history):
    review_chatbot_crew = Crew(agents=[review_chatbot_agent], tasks=[review_chatbot_task], verbose=True)
    
    response = review_chatbot_crew.kickoff({"user_question": message, "History": history})
    
    # Use regex to find all .png filenames in the saved_plots directory
    image_paths = re.findall(r'saved_plots/[^()]+\([^)]+\)_\d+_\d+\.png', response)
    
    if image_paths:
        # Get the last image path (in case there are multiple)
        print(image_paths)
        image_path = image_paths[-1]
        print(f"Image path: {image_path}")
        
        # Extract the text response by removing the image path
        #text_response = response.replace(image_path, '').strip()
        
        try:
            with Image.open(image_path) as img:
                buffer = io.BytesIO()
                img.save(buffer, format="PNG")
                encoded_image = base64.b64encode(buffer.getvalue()).decode()
            
            full_response = f"{response}\n\nHere's the requested graph:\n\n![Lab Test Graph](data:image/png;base64,{encoded_image})"
            
            history.append((message, full_response))
        except Exception as e:
            error_response = f"An error occurred while processing the graph: {str(e)}"
            history.append((message, f"{response}\n\n{error_response}"))
    else:
        history.append((message, response))
    
    return "", history

In [8]:
import gradio as gr

css = """
h1 {
    text-align: center;
    display:block;
}
"""

def create_patient_interface(num_patients):
    with gr.Blocks(fill_height=True, fill_width=True, css=css, theme=gr.themes.Soft(primary_hue=gr.themes.colors.blue, secondary_hue=gr.themes.colors.sky)) as demo:
        gr.Markdown("# Chat With Medical Records")
        gr.Markdown("Enter a member ID to get a detailed analysis of the patient's medical history.")
        
        with gr.Column(scale=6):          
            # Medical Chatbot Section - Outside of Patient Tabs
            #gr.Markdown("## Medical Chatbot")
            with gr.TabItem("Medical Chatbot"):
                chatbot = gr.Chatbot(label="Medical Chatbot", height=600)
                msg = gr.Textbox(label="Ask a question")
                clear = gr.Button("Clear Chat")

            # Chatbot events
            msg.submit(
                fn=chat,  
                inputs=[msg, chatbot], 
                outputs=[msg, chatbot]
            )

            clear.click(
                lambda: None, 
                None, 
                chatbot, 
                queue=False
            )

        return demo

# Create interface with 3 patients
demo = create_patient_interface(2)
demo.launch(share=True)

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
which: no node in (/home/jovyan/envs/llm_agents/bin:/miniconda3/condabin:/opt/mssql-tools/bin:/usr/local/apache-jmeter-5.4.3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/miniconda3/bin:/usr/lib/spark:/usr/lib/spark/bin:/usr/lib/jvm/java-1.8.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://7219bdaa902578413f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in